In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s3e23/sample_submission.csv
/kaggle/input/playground-series-s3e23/train.csv
/kaggle/input/playground-series-s3e23/test.csv


In [2]:
database = pd.read_csv('/kaggle/input/playground-series-s3e23/train.csv')
database = database.drop(['id'], axis = 1)

In [3]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.svm import SVC
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tqdm import tqdm

def capOutliers(data: pd.Series):
    """
    Cap values of a numeric series to upper and lower limits based on IQR.

    Args:
        data (pd.Series): A numeric pandas Series.

    Returns:
        pd.Series: The capped pandas Series.
    """
    # Calcula os quartis e o intervalo interquartil (IIQ)
    q1 = np.percentile(data, 25)
    q3 = np.percentile(data, 75)
    iqr = q3 - q1

    # Define os limites inferior e superior
    lower_limit = q1 - 1.5 * iqr
    upper_limit = q3 + 1.5 * iqr

    # Aplica os limites para capear os valores
    data_capped = data.copy()
    data_capped[data_capped > upper_limit] = upper_limit
    data_capped[data_capped < lower_limit] = lower_limit

    print(f"Coluna ajustada: ")
    print(f"Q1: {q1:.2f}, Q3: {q3:.2f}, IIQ: {iqr:.2f}")
    print(f"Limite Inferior: {lower_limit:.2f}, Limite Superior: {upper_limit:.2f}")
    print("-" * 50)

    return data_capped

def processOutliers(database: pd.DataFrame):
    """
    Processa e ajusta os outliers em colunas numéricas de um DataFrame.

    Args:
        database (pd.DataFrame): DataFrame contendo os dados.

    Returns:
        pd.DataFrame: DataFrame com outliers ajustados.
    """
    numeric_columns = database.select_dtypes(include=['number']).columns
    for column in numeric_columns:
        print(f"Ajustando outliers na coluna: {column}")
        database[column] = capOutliers(database[column])
    print("\nOutliers foram ajustados com sucesso.")
    return database

database = processOutliers(database)


Ajustando outliers na coluna: loc
Coluna ajustada: 
Q1: 13.00, Q3: 42.00, IIQ: 29.00
Limite Inferior: -30.50, Limite Superior: 85.50
--------------------------------------------------
Ajustando outliers na coluna: v(g)
Coluna ajustada: 
Q1: 2.00, Q3: 6.00, IIQ: 4.00
Limite Inferior: -4.00, Limite Superior: 12.00
--------------------------------------------------
Ajustando outliers na coluna: ev(g)
Coluna ajustada: 
Q1: 1.00, Q3: 3.00, IIQ: 2.00
Limite Inferior: -2.00, Limite Superior: 6.00
--------------------------------------------------
Ajustando outliers na coluna: iv(g)
Coluna ajustada: 
Q1: 1.00, Q3: 4.00, IIQ: 3.00
Limite Inferior: -3.50, Limite Superior: 8.50
--------------------------------------------------
Ajustando outliers na coluna: n
Coluna ajustada: 
Q1: 25.00, Q3: 111.00, IIQ: 86.00
Limite Inferior: -104.00, Limite Superior: 240.00
--------------------------------------------------
Ajustando outliers na coluna: v
Coluna ajustada: 
Q1: 97.67, Q3: 560.25, IIQ: 462.58
Lim

/tmp/ipykernel_17/1290964845.py:37: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '54.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data_capped[data_capped > upper_limit] = upper_limit
/tmp/ipykernel_17/1290964845.py:37: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data_capped[data_capped > upper_limit] = upper_limit


In [4]:
def make_predictions(model, X_test, test_ids):
    predictions = model.predict_proba(X_test)[:, 1]
    submission_df = pd.DataFrame({
        'id': test_ids,
        'defects': predictions
    })
    return submission_df
    
# Função para processar os outliers (ajustar conforme sua necessidade)
def processOutliers(database):
    # Aqui você pode aplicar a remoção de outliers com base nos limites superiores e inferiores
    Q3 = np.percentile(database, 75)
    Q1 = np.percentile(database, 25)
    IIQ = Q3 - Q1
    LS = Q3 + (1.5 * IIQ)
    LI = Q1 - (1.5 * IIQ)
    
    # Substituindo os outliers pelos limites superiores/inferiores
    database = np.where(database > LS, LS, database)
    database = np.where(database < LI, LI, database)
    
    return database

# Carregar e processar os dados
def loadAndPreprocessData(file_path):
    # Carregar os dados
    database = pd.read_csv(file_path)
    
    # Processar outliers (ajustado para o dataset completo, incluindo features)
    X = database.drop(columns=['defects'])
    y = database['defects']
    
    # Processar os dados de entrada
    X = processOutliers(X)
    
    # Dividir o dataset em treino e teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Escalonamento dos dados
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    return X_train, X_test, y_train, y_test

# Modelo 1: Regressão Logística com tqdm
def trainLogisticRegression(X_train, y_train):
    print("Treinando Regressão Logística...")
    model = LogisticRegression(max_iter=1000)
    
    # Simulando barra de progresso para treinar em etapas
    for _ in tqdm(range(1), desc="Treinando"):
        model.fit(X_train, y_train)
    return model

# Modelo 3: Random Forest com tqdm
def trainRandomForest(X_train, y_train):
    print("Treinando Random Forest...")
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    for _ in tqdm(range(1), desc="Treinando"):
        model.fit(X_train, y_train)
    return model

# Modelo 4: MLP com 256 camadas e callback de progresso
def trainMLP(X_train, y_train, input_dim):
    print("Treinando MLP com 256 camadas...")
    
    # Ajuste para usar a camada de entrada corretamente
    model = Sequential()
    model.add(Input(shape=(input_dim,)))  # Definindo a camada de entrada
    
    for _ in range(3):
        model.add(Dense(16, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    # Barra de progresso para o treinamento
    model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)
    
    return model

# Avaliação de modelos
def evaluateModel(model, X_test, y_test, isMLP=False):
    if isMLP:
        y_pred = (model.predict(X_test) > 0.5).astype("int32")
    else:
        y_pred = model.predict(X_test)
    print("Acurácia:", accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))



# Função para fazer as predições e salvar a submissão
def make_predictions(model, X_test, test_ids):
    # Gerar as predições
    predictions = model.predict(X_test)[:, 0]  # Ou model.predict_proba(X_test)[:, 1] se o modelo retornar probabilidades
    # Criar o DataFrame com as predições
    submission_df = pd.DataFrame({
        'id': test_ids,
        'defects': predictions
    })
    return submission_df

# Função para salvar o modelo treinado
def save_trained_model(model, model_name='trained_model.h5'):
    # Salvar o modelo em um arquivo .h5
    model.save(model_name)
    print(f"Modelo salvo como {model_name}")

# Função para carregar o modelo salvo
def load_trained_model(model_name='trained_model.h5'):
    # Carregar o modelo salvo
    model = load_model(model_name)
    print(f"Modelo carregado: {model_name}")
    return model

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, classification_report
import pickle

# Função para carregar e preparar os dados
def load_and_prepare_data(train_path, test_path):
    train_data = pd.read_csv(train_path)
    test_data = pd.read_csv(test_path)
    
    X = train_data.drop(['id', 'defects'], axis=1)
    y = train_data['defects']
    X_test = test_data.drop(['id'], axis=1)
    
    return X, y, X_test, test_data['id']

# Função para treinar o modelo (classificador)
def train_model(X, y, model_type='logistic_regression'):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
    
    if model_type == 'logistic_regression':
        model = LogisticRegression(max_iter=1000)
    elif model_type == 'random_forest':
        model = RandomForestClassifier(n_estimators=100, random_state=42)
    elif model_type == 'decision_tree':
        model = DecisionTreeClassifier(random_state=42)
    elif model_type == 'mlp':
        model = Sequential()
        model.add(Input(shape=(X_train.shape[1],)))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(1, activation='sigmoid'))
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    if model_type == 'mlp':
        # Adicionando EarlyStopping
        early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
        model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_val, y_val), 
                  verbose=1, callbacks=[early_stopping])
    else:
        model.fit(X_train, y_train)
    
    # Avaliação do modelo (no conjunto de validação)
    if model_type == 'mlp':
        y_pred = (model.predict(X_val) > 0.5).astype("int32")
    else:
        y_pred = model.predict(X_val)
    
    accuracy = accuracy_score(y_val, y_pred)
    print(f"Modelo {model_type} - Acurácia no conjunto de validação: {accuracy:.4f}")
    
    return model

# Função para gerar predições e salvar em CSV
def make_predictions(model, X_test, test_ids):
    if isinstance(model, Sequential):  # MLP (Keras)
        predictions = model.predict(X_test)
        predictions = (predictions > 0.5).astype("int32")[:, 0]
    else:  # Modelos clássicos como LogisticRegression, RandomForest, DecisionTree
        predictions = model.predict_proba(X_test)[:, 1]
    
    submission_df = pd.DataFrame({
        'id': test_ids,
        'defects': predictions
    })
    return submission_df

# Função para salvar o modelo
def save_model(model, model_name):
    with open(f'{model_name}.pkl', 'wb') as file:
        pickle.dump(model, file)
    print(f"Modelo {model_name} salvo em {model_name}.pkl")

# Função principal
def main():
    train_path = '/kaggle/input/playground-series-s3e23/train.csv'
    test_path = '/kaggle/input/playground-series-s3e23/test.csv'
    
    # Carregar dados
    X, y, X_test, test_ids = load_and_prepare_data(train_path, test_path)
    
    # Normalizar os dados
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    X_test = scaler.transform(X_test)
    
    # Treinar e fazer predições com diferentes modelos
    models = ['logistic_regression', 'random_forest', 'decision_tree', 'mlp']
    
    for model_type in models:
        print(f"\nTreinando o modelo: {model_type}")
        
        model = train_model(X, y, model_type=model_type)
        
        # Salvar o modelo
        save_model(model, model_type)
        
        # Gerar predições e salvar arquivo de submissão
        submission_df = make_predictions(model, X_test, test_ids)
        submission_df.to_csv(f'{model_type}_submission.csv', index=False)
        print(f"Submissão salva como '{model_type}_submission.csv'")

if __name__ == "__main__":
    main()



Treinando o modelo: logistic_regression
Modelo logistic_regression - Acurácia no conjunto de validação: 0.8098
Modelo logistic_regression salvo em logistic_regression.pkl
Submissão salva como 'logistic_regression_submission.csv'

Treinando o modelo: random_forest
Modelo random_forest - Acurácia no conjunto de validação: 0.8093
Modelo random_forest salvo em random_forest.pkl
Submissão salva como 'random_forest_submission.csv'

Treinando o modelo: decision_tree
Modelo decision_tree - Acurácia no conjunto de validação: 0.7172
Modelo decision_tree salvo em decision_tree.pkl
Submissão salva como 'decision_tree_submission.csv'

Treinando o modelo: mlp
Epoch 1/100
2545/2545 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.8096 - loss: 0.4474 - val_accuracy: 0.8133 - val_loss: 0.4333
Epoch 2/100
2545/2545 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.8090 - loss: 0.4385 - val_accuracy: 0.8136 - val_loss: 0.4334
Epoch 3/100
2545/2545 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.8142 - loss: 0